In [9]:
from pathlib import Path
import pandas as pd
from rdflib import Graph, URIRef
from rdflib.namespace import OWL, RDFS, XSD

# Start from the notebook's current directory
NB_DIR = Path.cwd().resolve()

# Walk up to find a directory that contains a 'src' folder
SRC_DIR = None
for p in [NB_DIR] + list(NB_DIR.parents):
    candidate = p / "assignment" / "src"
    if candidate.exists():
        SRC_DIR = candidate
        break
    candidate = p / "src"  # also allow projects that don't have the extra 'assignment' level
    if candidate.exists():
        SRC_DIR = candidate
        break

if SRC_DIR is None:
    raise FileNotFoundError(
        f"Could not find a 'src' directory above {NB_DIR}. "
        "If your project root is at e.g. .../assignment/, open the notebook there or set SRC_DIR manually."
    )


DATA_DIR = SRC_DIR / "data"
# Create the data directory if it doesn't exist
DATA_DIR.mkdir(exist_ok=True)

print("Notebook CWD:   ", NB_DIR)
print("Resolved SRC_DIR:", SRC_DIR)
print("Resolved DATA_DIR:", DATA_DIR)
print("TTL files found:", [p.name for p in SRC_DIR.glob("*.ttl")])


TTL = {
    "bfo":  SRC_DIR / "bfo-core.ttl",
    "ies":  SRC_DIR / "ies.ttl",
    "qudt": SRC_DIR / "qudt.ttl",
    "ccom": SRC_DIR / "ccom.ttl",
    "ccot": SRC_DIR / "ccot.ttl",
    "to":   SRC_DIR / "time.ttl",
}

OUT = {
    "bfo":  DATA_DIR / "bfo-axioms.xlsx",
    "ies":  DATA_DIR / "ies-axioms.xlsx",
    "qudt": DATA_DIR / "qudt-axioms.xlsx",
    "ccom": DATA_DIR / "ccom-axioms.xlsx",
    "ccot": DATA_DIR / "ccot-axioms.xlsx",
    "to":   DATA_DIR / "to-axioms.xlsx",
}

CLASS_AXIOM_QUERY = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl:  <http://www.w3.org/2002/07/owl#>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc:   <http://purl.org/dc/elements/1.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>

SELECT ?iri ?label ?axiom
WHERE {
  # Get all classes
  ?iri a ?classType .
  VALUES ?classType { owl:Class rdfs:Class }
  
  # Get label (prefer rdfs:label, fall back to skos:prefLabel)
  OPTIONAL { 
    ?iri rdfs:label ?rdfsLabel 
  }
  OPTIONAL { 
    ?iri skos:prefLabel ?skosLabel 
  }
  BIND(COALESCE(?rdfsLabel, ?skosLabel, "") AS ?label)
  
  # Filter out deprecated classes
  FILTER NOT EXISTS {
    ?iri owl:deprecated ?d .
    FILTER(?d = true || ?d = "true"^^xsd:string)
  }
  
  # Get axioms - union of multiple axiom types
  {
    # SubClassOf axioms
    ?iri rdfs:subClassOf ?target .
    FILTER(!isBlank(?target))
    BIND(CONCAT("SubClassOf: ", STR(?target)) AS ?axiom)
  } UNION {
    # DisjointWith axioms
    ?iri owl:disjointWith ?target .
    FILTER(!isBlank(?target))
    BIND(CONCAT("DisjointWith: ", STR(?target)) AS ?axiom)
  } UNION {
    # EquivalentClass axioms
    ?iri owl:equivalentClass ?target .
    FILTER(!isBlank(?target))
    BIND(CONCAT("EquivalentClass: ", STR(?target)) AS ?axiom)
  } UNION {
    # Domain axioms (properties that have this class as domain)
    ?prop rdfs:domain ?iri .
    BIND(CONCAT("Domain of: ", STR(?prop)) AS ?axiom)
  } UNION {
    # Range axioms (properties that have this class as range)
    ?prop rdfs:range ?iri .
    BIND(CONCAT("Range of: ", STR(?prop)) AS ?axiom)
  } UNION {
    # Comments
    ?iri rdfs:comment ?comment .
    BIND(CONCAT("Comment: ", STR(?comment)) AS ?axiom)
  } UNION {
    # Descriptions (dc or dcterms)
    {
      ?iri dc:description ?desc .
      BIND(CONCAT("Description: ", STR(?desc)) AS ?axiom)
    } UNION {
      ?iri dcterms:description ?desc .
      BIND(CONCAT("Description: ", STR(?desc)) AS ?axiom)
    }
  } UNION {
    # Identifiers (for IES)
    ?iri dcterms:identifier ?identifier .
    BIND(CONCAT("Identifier: ", STR(?identifier)) AS ?axiom)
  } UNION {
    # Definitions (for CCO)
    ?iri skos:definition ?def .
    BIND(CONCAT("Definition: ", STR(?def)) AS ?axiom)
  }
}
ORDER BY ?iri ?axiom
"""
def run_local_select_axioms(ttl_path: Path, query: str) -> pd.DataFrame:
    g = Graph()
    g.parse(ttl_path)  # rdflib guesses Turtle from .ttl
    res = g.query(query)
    rows = []
    for row in res:
        d = row.asdict()  # {'iri': rdflib.term.URIRef(...), 'label': rdflib.term.Literal(...)} or missing
        iri_term   = d.get("iri")
        label_term = d.get("label")
        axiom_term = d.get("axiom")
        rows.append({
            "iri":   str(iri_term)   if iri_term is not None else None,
            "label": str(label_term) if label_term is not None else None,
            "axiom": str(axiom_term) if label_term is not None else None,
        })
    return pd.DataFrame(rows, columns=["iri", "label","axiom"])

for key, ttl_path in TTL.items():
    if not ttl_path.exists():
        print(f"⚠️  Missing {ttl_path} — skipping {key}")
        continue
    print(f"Querying {ttl_path} …")
    df = run_local_select_axioms(ttl_path, CLASS_AXIOM_QUERY)
    df.to_excel(OUT[key], index=False)
    print(f"✅ Saved {len(df)} rows to {OUT[key]}")

Notebook CWD:    C:\Users\crist\Documents\GitHub\Ontology-Tradecraft\Ontology-Tradecraft\projects\project-3\assignment\notebooks
Resolved SRC_DIR: C:\Users\crist\Documents\GitHub\Ontology-Tradecraft\Ontology-Tradecraft\projects\project-3\assignment\src
Resolved DATA_DIR: C:\Users\crist\Documents\GitHub\Ontology-Tradecraft\Ontology-Tradecraft\projects\project-3\assignment\src\data
TTL files found: ['bfo-core.ttl', 'ccom.ttl', 'ccot.ttl', 'ies.ttl', 'qudt.ttl', 'time.ttl']
Querying C:\Users\crist\Documents\GitHub\Ontology-Tradecraft\Ontology-Tradecraft\projects\project-3\assignment\src\bfo-core.ttl …
✅ Saved 158 rows to C:\Users\crist\Documents\GitHub\Ontology-Tradecraft\Ontology-Tradecraft\projects\project-3\assignment\src\data\bfo-axioms.xlsx
Querying C:\Users\crist\Documents\GitHub\Ontology-Tradecraft\Ontology-Tradecraft\projects\project-3\assignment\src\ies.ttl …
✅ Saved 1929 rows to C:\Users\crist\Documents\GitHub\Ontology-Tradecraft\Ontology-Tradecraft\projects\project-3\assignment